In [1]:
import numpy as np

def fullfact(levels):
    """
    Create a general full-factorial design
    
    Parameters
    ----------
    levels : array-like
        An array of integers that indicate the number of levels of each input
        design factor.
    
    Returns
    -------
    mat : 2d-array
        The design matrix with coded levels 0 to k-1 for a k-level factor
    
    Example
    -------
    ::
    
        >>> fullfact([2, 4, 3])
        array([[ 0.,  0.,  0.],
               [ 1.,  0.,  0.],
               [ 0.,  1.,  0.],
               [ 1.,  1.,  0.],
               [ 0.,  2.,  0.],
               [ 1.,  2.,  0.],
               [ 0.,  3.,  0.],
               [ 1.,  3.,  0.],
               [ 0.,  0.,  1.],
               [ 1.,  0.,  1.],
               [ 0.,  1.,  1.],
               [ 1.,  1.,  1.],
               [ 0.,  2.,  1.],
               [ 1.,  2.,  1.],
               [ 0.,  3.,  1.],
               [ 1.,  3.,  1.],
               [ 0.,  0.,  2.],
               [ 1.,  0.,  2.],
               [ 0.,  1.,  2.],
               [ 1.,  1.,  2.],
               [ 0.,  2.,  2.],
               [ 1.,  2.,  2.],
               [ 0.,  3.,  2.],
               [ 1.,  3.,  2.]])
               
    """
    n = len(levels)  # number of factors
    nb_lines = np.prod(levels)  # number of trial conditions
    H = np.zeros((nb_lines, n))
    
    level_repeat = 1
    range_repeat = np.prod(levels)
    for i in range(n):
        range_repeat /= levels[i]
        lvl = []
        for j in range(levels[i]):
            lvl += [j]*level_repeat
        rng = lvl*int(range_repeat)
        level_repeat *= levels[i]
        H[:, i] = rng
     
    return H

def ff2n(n):
    """
    Create a 2-Level full-factorial design
    
    Parameters
    ----------
    n : int
        The number of factors in the design.
    
    Returns
    -------
    mat : 2d-array
        The design matrix with coded levels -1 and 1
    
    Example
    -------
    ::
    
        >>> ff2n(3)
        array([[-1., -1., -1.],
               [ 1., -1., -1.],
               [-1.,  1., -1.],
               [ 1.,  1., -1.],
               [-1., -1.,  1.],
               [ 1., -1.,  1.],
               [-1.,  1.,  1.],
               [ 1.,  1.,  1.]])
       
    """
    return 2*fullfact([2]*n) - 1


from pyDOE.doe_star import star
from pyDOE.doe_union import union
from pyDOE.doe_repeat_center import repeat_center

def ccdesign(n, center=(4, 4), alpha='orthogonal', face='circumscribed'):
    """
    Central composite design
    
    Parameters
    ----------
    n : int
        The number of factors in the design.
    
    Optional
    --------
    center : int array
        A 1-by-2 array of integers, the number of center points in each block
        of the design. (Default: (4, 4)).
    alpha : str
        A string describing the effect of alpha has on the variance. ``alpha``
        can take on the following values:
        
        1. 'orthogonal' or 'o' (Default)
        
        2. 'rotatable' or 'r'
        
    face : str
        The relation between the start points and the corner (factorial) points.
        There are three options for this input:
        
        1. 'circumscribed' or 'ccc': This is the original form of the central
           composite design. The star points are at some distance ``alpha``
           from the center, based on the properties desired for the design.
           The start points establish new extremes for the low and high
           settings for all factors. These designs have circular, spherical,
           or hyperspherical symmetry and require 5 levels for each factor.
           Augmenting an existing factorial or resolution V fractional 
           factorial design with star points can produce this design.
        
        2. 'inscribed' or 'cci': For those situations in which the limits
           specified for factor settings are truly limits, the CCI design
           uses the factors settings as the star points and creates a factorial
           or fractional factorial design within those limits (in other words,
           a CCI design is a scaled down CCC design with each factor level of
           the CCC design divided by ``alpha`` to generate the CCI design).
           This design also requires 5 levels of each factor.
        
        3. 'faced' or 'ccf': In this design, the star points are at the center
           of each face of the factorial space, so ``alpha`` = 1. This 
           variety requires 3 levels of each factor. Augmenting an existing 
           factorial or resolution V design with appropriate star points can 
           also produce this design.
    
    Notes
    -----
    - Fractional factorial designs are not (yet) available here.
    - 'ccc' and 'cci' can be rotatable design, but 'ccf' cannot.
    - If ``face`` is specified, while ``alpha`` is not, then the default value
      of ``alpha`` is 'orthogonal'.
        
    Returns
    -------
    mat : 2d-array
        The design matrix with coded levels -1 and 1
    
    Example
    -------
    ::
    
        >>> ccdesign(3)
        array([[-1.        , -1.        , -1.        ],
               [ 1.        , -1.        , -1.        ],
               [-1.        ,  1.        , -1.        ],
               [ 1.        ,  1.        , -1.        ],
               [-1.        , -1.        ,  1.        ],
               [ 1.        , -1.        ,  1.        ],
               [-1.        ,  1.        ,  1.        ],
               [ 1.        ,  1.        ,  1.        ],
               [ 0.        ,  0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        ],
               [-1.82574186,  0.        ,  0.        ],
               [ 1.82574186,  0.        ,  0.        ],
               [ 0.        , -1.82574186,  0.        ],
               [ 0.        ,  1.82574186,  0.        ],
               [ 0.        ,  0.        , -1.82574186],
               [ 0.        ,  0.        ,  1.82574186],
               [ 0.        ,  0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        ],
               [ 0.        ,  0.        ,  0.        ]])
        
       
    """
    # Check inputs
    assert isinstance(n, int) and n>1, '"n" must be an integer greater than 1.'
    assert alpha.lower() in ('orthogonal', 'o', 'rotatable', 
        'r'), 'Invalid value for "alpha": {:}'.format(alpha)
    assert face.lower() in ('circumscribed', 'ccc', 'inscribed', 'cci',
        'faced', 'ccf'), 'Invalid value for "face": {:}'.format(face)
    
    try:
        nc = len(center)
    except:
        raise TypeError('Invalid value for "center": {:}. Expected a 1-by-2 array.'.format(center))
    else:
        if nc!=2:
            raise ValueError('Invalid number of values for "center" (expected 2, but got {:})'.format(nc))

    # Orthogonal Design
    if alpha.lower() in ('orthogonal', 'o'):
        H2, a = star(n, alpha='orthogonal', center=center)
    
    # Rotatable Design
    if alpha.lower() in ('rotatable', 'r'):
        H2, a = star(n, alpha='rotatable')
    
    # Inscribed CCD
    if face.lower() in ('inscribed', 'cci'):
        H1 = ff2n(n)
        H1 = H1/a  # Scale down the factorial points
        H2, a = star(n)
    
    # Faced CCD
    if face.lower() in ('faced', 'ccf'):
        H2, a = star(n)  # Value of alpha is always 1 in Faced CCD
        H1 = ff2n(n)
    
    # Circumscribed CCD
    if face.lower() in ('circumscribed', 'ccc'):
        H1 = ff2n(n)
    
    C1 = repeat_center(n, center[0])
    C2 = repeat_center(n, center[1])

    H1 = union(H1, C1)
    H2 = union(H2, C2)
    H = union(H1, H2)

    return H

In [2]:
fullfact([2,2,2,2])

array([[0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 1., 0.],
       [1., 1., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 1.],
       [0., 1., 0., 1.],
       [1., 1., 0., 1.],
       [0., 0., 1., 1.],
       [1., 0., 1., 1.],
       [0., 1., 1., 1.],
       [1., 1., 1., 1.]])

In [3]:
ff2n(4)

array([[-1., -1., -1., -1.],
       [ 1., -1., -1., -1.],
       [-1.,  1., -1., -1.],
       [ 1.,  1., -1., -1.],
       [-1., -1.,  1., -1.],
       [ 1., -1.,  1., -1.],
       [-1.,  1.,  1., -1.],
       [ 1.,  1.,  1., -1.],
       [-1., -1., -1.,  1.],
       [ 1., -1., -1.,  1.],
       [-1.,  1., -1.,  1.],
       [ 1.,  1., -1.,  1.],
       [-1., -1.,  1.,  1.],
       [ 1., -1.,  1.,  1.],
       [-1.,  1.,  1.,  1.],
       [ 1.,  1.,  1.,  1.]])

In [4]:
ccd_levels = ccdesign(5, center=(0,1), alpha='o', face='ccc')
ccd_levels

array([[-1.        , -1.        , -1.        , -1.        , -1.        ],
       [ 1.        , -1.        , -1.        , -1.        , -1.        ],
       [-1.        ,  1.        , -1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        , -1.        , -1.        ],
       [-1.        , -1.        ,  1.        , -1.        , -1.        ],
       [ 1.        , -1.        ,  1.        , -1.        , -1.        ],
       [-1.        ,  1.        ,  1.        , -1.        , -1.        ],
       [ 1.        ,  1.        ,  1.        , -1.        , -1.        ],
       [-1.        , -1.        , -1.        ,  1.        , -1.        ],
       [ 1.        , -1.        , -1.        ,  1.        , -1.        ],
       [-1.        ,  1.        , -1.        ,  1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ,  1.        , -1.        ],
       [-1.        , -1.        ,  1.        ,  1.        , -1.        ],
       [ 1.        , -1.        ,  1. 

In [5]:
levels = np.array([-2.34520788,-1,0,1,2.34520788])

# variables ranges
Thickness = (0.001, 0.010)
CP3_y = (-0.01255805, 0.0)
T0in = (400, 800)
p0in = (4.0e5, 1.0e6)
outerTemperature = (290, 400)

Thickness_levels = np.interp(levels, (levels.min(), levels.max()), Thickness)
CP3_y_levels = np.interp(levels, (levels.min(), levels.max()), CP3_y)
T0in_levels = np.interp(levels, (levels.min(), levels.max()), T0in)
p0in_lebels = np.interp(levels, (levels.min(), levels.max()), p0in)
outerTemperature_levels = np.interp(levels, (levels.min(), levels.max()), outerTemperature)

In [6]:
converted_levels = np.array(
    [Thickness_levels,
     CP3_y_levels,
     T0in_levels,
     p0in_lebels,
     outerTemperature_levels,]
)
converted_levels.shape
converted_levels[0]

array([0.001     , 0.00358119, 0.0055    , 0.00741881, 0.01      ])

In [7]:
Thickness_levels

array([0.001     , 0.00358119, 0.0055    , 0.00741881, 0.01      ])

In [8]:
levels*(Thickness_levels[-2]-np.mean(Thickness_levels)) + np.mean(Thickness_levels)

array([0.001     , 0.00358119, 0.0055    , 0.00741881, 0.01      ])

In [9]:
ccd_levels_converted = np.copy(ccd_levels)
for i in range(ccd_levels.shape[1]):
    ccd_levels_converted[:,i] = ccd_levels[:,i]*(converted_levels[i][-2]-np.mean(converted_levels[i])) + np.mean(converted_levels[i])


In [16]:
np.set_printoptions(precision=5, suppress=True)
print(ccd_levels_converted)

[[     0.00358     -0.00896    514.71971 572079.57019    321.54792]
 [     0.00742     -0.00896    514.71971 572079.57019    321.54792]
 [     0.00358     -0.0036     514.71971 572079.57019    321.54792]
 [     0.00742     -0.0036     514.71971 572079.57019    321.54792]
 [     0.00358     -0.00896    685.28029 572079.57019    321.54792]
 [     0.00742     -0.00896    685.28029 572079.57019    321.54792]
 [     0.00358     -0.0036     685.28029 572079.57019    321.54792]
 [     0.00742     -0.0036     685.28029 572079.57019    321.54792]
 [     0.00358     -0.00896    514.71971 827920.42981    321.54792]
 [     0.00742     -0.00896    514.71971 827920.42981    321.54792]
 [     0.00358     -0.0036     514.71971 827920.42981    321.54792]
 [     0.00742     -0.0036     514.71971 827920.42981    321.54792]
 [     0.00358     -0.00896    685.28029 827920.42981    321.54792]
 [     0.00742     -0.00896    685.28029 827920.42981    321.54792]
 [     0.00358     -0.0036     685.28029 827920.

In [19]:
np.savetxt('ccd_experiment.txt' ,ccd_levels_converted)